## Week 4, Lab 1: Predicting Left-Handedness from Psychological Factors
> Author: Matt Brems

We can sketch out the data science process as follows:
1. Define the problem.
2. Obtain the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

We'll walk through a full data science problem in this lab. 

---
## Step 1: Define The Problem.

You're currently a data scientist working at a university. A professor of psychology is attempting to study the relationship between personalities and left-handedness. They have tasked you with gathering evidence so that they may publish.

Specifically, the professor says "I need to prove that left-handedness is caused by some personality trait. Go find that personality trait and the data to back it up."

As a data scientist, you know that any real data science problem must be **specific** and **conclusively answerable**. For example:
- Bad data science problem: "What is the link between obesity and blood pressure?"
    - This is vague and is not conclusively answerable. That is, two people might look at the conclusion and one may say "Sure, the problem has been answered!" and the other may say "The problem has not yet been answered."
- Good data science problem: "Does an association exist between obesity and blood pressure?"
    - This is more specific and is conclusively answerable. The problem specifically is asking for a "Yes" or "No" answer. Based on that, two independent people should both be able to say either "Yes, the problem has been answered" or "No, the problem has not yet been answered."
- Excellent data science problem: "As obesity increases, how does blood pressure change?"
    - This is very specific and is conclusively answerable. The problem specifically seeks to understand the effect of one variable on the other.

### 1. In the context of the left-handedness and personality example, what are three specific and conclusively answerable problems that you could answer using data science? 

> You might find it helpful to check out the codebook in the repo for some inspiration.

Answer:  
How does probability of left-handedness changes with inclination to decorate things?  
How does probability of left-handedness changes with ability to remember birthdays?  
How does probability of left-handedness changes with tendency to give people handmade gifts?

---
## Step 2: Obtain the data.

### 2. Read in the file titled "data.csv."
> Hint: Despite being saved as a .csv file, you won't be able to simply `pd.read_csv()` this data!

In [1]:
import pandas as pd
import plotly.express as px
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score

In [2]:
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)

In [3]:
df=pd.read_csv("data.csv",sep='\t')

### 3. Suppose that, instead of us giving you this data in a file, you were actually conducting a survey to gather this data yourself. From an ethics/privacy point of view, what are three things you might consider when attempting to gather this data?
> When working with sensitive data like sexual orientation or gender identity, we need to consider how this data could be used if it fell into the wrong hands!

Answer:  
Anonymize any personal data identifier, name, ID number

---
## Step 3: Explore the data.

### 4. Conduct exploratory data analysis on this dataset.
> If you haven't already, be sure to check out the codebook in the repo, as that will help in your EDA process.

In [4]:
#replace "both-handedness" as "left-handedness"
df['hand']=df['hand'].replace(3,2)
#drop hand==0
df=df[~(df['hand']==0)]

In [5]:
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)

In [6]:
#one hot encode a few fields
ohe_list=['engnat','education','gender','orientation','race','religion','fromgoogle','country']
df=pd.get_dummies(df,prefix=ohe_list,columns=ohe_list,drop_first=True)

In [7]:
df.corr()['hand'].sort_values()

gender_2        -0.058587
engnat_2        -0.052819
orientation_1   -0.048535
country_AU      -0.038409
Q8              -0.034042
Q23             -0.033173
Q20             -0.032990
Q22             -0.032774
race_7          -0.030348
religion_5      -0.028037
country_HK      -0.027781
country_BR      -0.027781
country_IN      -0.027379
race_2          -0.027283
country_FI      -0.027092
Q18             -0.026840
Q6              -0.026208
country_PL      -0.022624
Q14             -0.021733
country_CL      -0.021699
Q40             -0.021517
country_DE      -0.021365
Q2              -0.021204
Q10             -0.020887
Q24             -0.020780
country_MX      -0.020687
country_JO      -0.020687
religion_3      -0.019984
introelapse     -0.019016
testelapse      -0.018841
country_RO      -0.018498
country_EE      -0.018498
country_AT      -0.017574
country_KR      -0.017301
country_RU      -0.017301
country_ID      -0.016493
country_AR      -0.016016
country_MY      -0.016016
country_NL  

The most correlating feature is merely a little more than 5% correlation.

---
## Step 4: Model the data.

### 5. Suppose I wanted to use Q1 - Q44 to predict whether or not the person is left-handed. Would this be a classification or regression problem? Why?

Answer: Classification. The final predictor label is a boolean.

### 6. We want to use $k$-nearest neighbors to predict whether or not a person is left-handed based on their responses to Q1 - Q44. Before doing that, however, you remember that it is often a good idea to standardize your variables. In general, why would we standardize our variables? Give an example of when we would standardize our variables.

Answer:  
Standardization allows easier treatement for features with different ranges.  
Usually you'd standardised a feature that is known to have Gaussian distribution.

### 7. Give an example of when we might not standardize our variables.

Answer: When your data is not numerical, or of unknown distribution.

### 8. Based on your answers to 6 and 7, do you think we should standardize our predictor variables in this case? Why or why not?

Answer: Probably no need, since the responses across the questions are of a fixed range 1 to 5.

### 9. We want to use $k$-nearest neighbors to predict whether or not a person is left-handed. What munging/cleaning do we need to do to our $y$ variable in order to explicitly answer this question? Do it.

Answer: Features should have no nulls, and all should be numerical.

In [8]:
#Find columnes with null. Should show up empty.
df.columns[df.isnull().sum()==True]

Index([], dtype='object')

In [9]:
#check for non-int
df.dtypes

Q1               int64
Q2               int64
Q3               int64
Q4               int64
Q5               int64
Q6               int64
Q7               int64
Q8               int64
Q9               int64
Q10              int64
Q11              int64
Q12              int64
Q13              int64
Q14              int64
Q15              int64
Q16              int64
Q17              int64
Q18              int64
Q19              int64
Q20              int64
Q21              int64
Q22              int64
Q23              int64
Q24              int64
Q25              int64
Q26              int64
Q27              int64
Q28              int64
Q29              int64
Q30              int64
Q31              int64
Q32              int64
Q33              int64
Q34              int64
Q35              int64
Q36              int64
Q37              int64
Q38              int64
Q39              int64
Q40              int64
Q41              int64
Q42              int64
Q43              int64
Q44        

### 10. The professor for whom you work suggests that you set $k = 4$. In this specific case, why might this be a bad idea?

Answer: Only consindering the nearest 4 neighbours is rather myopic. The model should look at more neighbours to generalize better.

### 11. Let's *(finally)* use $k$-nearest neighbors to predict whether or not a person is left-handed!

> Be sure to create a train/test split with your data!

> Create four separate models, one with $k = 3$, one with $k = 5$, one with $k = 15$, and one with $k = 25$.

> Instantiate and fit your models.

In [10]:
# Set up X and y
X=df.iloc[:,0:44]
y=df['hand']

In [11]:
y.value_counts()

1    3542
2     631
Name: hand, dtype: int64

In [12]:
# Conduct a train/test split.
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.25,
                                                    random_state = 42)

In [13]:
n_neighbors=[3,5,15,25]

In [14]:
#reset before every run
knn_models=[]

for i in range(len(n_neighbors)):
    knn=KNeighborsClassifier(n_neighbors=n_neighbors[i])
    knn.fit(X_train,y_train)
    knn_models.append(knn)

Being good data scientists, we know that we might not run just one type of model. We might run many different models and see which is best.

### 12. We want to use logistic regression to predict whether or not a person is left-handed. Before we do that, let's check the [documentation for logistic regression in sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). Is there default regularization? If so, what is it? If not, how do you know?

Answer: Yes, it is applied by default. L1 and L2 regularization are both supported. Elastic Net is only supported by `saga` solver.

### 13. We want to use logistic regression to predict whether or not a person is left-handed. Before we do that, should we standardize our features?

Answer: Standardization is not required for logistic regression.

### 14. Let's use logistic regression to predict whether or not the person is left-handed.


> Be sure to use the same train/test split with your data as with your $k$-NN model above!

> Create four separate models, one with LASSO and $\alpha = 1$, one with LASSO and $\alpha = 10$, one with Ridge and $\alpha = 1$, and one with Ridge and $\alpha = 10$. *(Hint: Be careful with how you specify $\alpha$ in your model!)*

> Instantiate and fit your models.

In [15]:
penalties=['l1','l2']
alphas=[1,10]

In [16]:
#reset before every run
logreg_models=[]

for p in penalties:
    for a in alphas:
        logreg=LogisticRegression(penalty=p,C=1/a,solver='liblinear')
        logreg.fit(X_train,y_train)
        display(logreg)
        display(logreg.coef_.mean())
        logreg_models.append(logreg)

LogisticRegression(penalty='l1', solver='liblinear')

-0.007448857943865028

LogisticRegression(C=0.1, penalty='l1', solver='liblinear')

-0.011336064057686036

LogisticRegression(solver='liblinear')

-0.006749894507787276

LogisticRegression(C=0.1, solver='liblinear')

-0.00943170403922317

---
## Step 5: Evaluate the model(s).

### 15. Before calculating any score on your data, take a step back. Think about your $X$ variable and your $Y$ variable. Do you think your $X$ variables will do a good job of predicting your $Y$ variable? Why or why not? What impact do you think this will have on your scores?

Answer: Unable to tell. Correlation feels low.

### 16. Using accuracy as your metric, evaluate all eight of your models on both the training and testing sets. Put your scores below. (If you want to be fancy and generate a table in Markdown, there's a [Markdown table generator site linked here](https://www.tablesgenerator.com/markdown_tables#).)
- Note: Your answers here might look a little weird. You didn't do anything wrong; that's to be expected!

In [17]:
for model in [*knn_models, *logreg_models]:
    display(model.score(X_train,y_train))

0.8705656759348035

0.8497922658996484

0.8494726749760306

0.849153084052413

0.849153084052413

0.8494726749760306

0.849153084052413

0.8494726749760306

In [18]:
for model in [*knn_models, *logreg_models]:
    display(model.score(X_test,y_test))

0.8007662835249042

0.8247126436781609

0.8477011494252874

0.8477011494252874

0.8477011494252874

0.8467432950191571

0.8477011494252874

0.8477011494252874

Answer: It does seems that all the logistic regression models perform equally well. Knn classifier with $k\ge15$ plateaus in accuracy and is marginally better than the logistic regression models.

### 17. In which of your $k$-NN models is there evidence of overfitting? How do you know?

Answer: First 2 models. $k=3$ and $k=5$. Training accuracy is higher than test accuracy by quite a bit, and since there is room for improvement with bigger $k$, we can deduce that these 2 were overfitted.

### 18. Broadly speaking, how does the value of $k$ in $k$-NN affect the bias-variance tradeoff? (i.e. As $k$ increases, how are bias and variance affected?)

Answer: as K increases, bias increases but variance decreases.

### 19. If you have a $k$-NN model that has evidence of overfitting, what are three things you might try to do to combat overfitting?

Answer: increase k, reduce number of features, increase number or rows.

### 20. In which of your logistic regression models is there evidence of overfitting? How do you know?

Answer: All of them. The accuracy for training set is higher than test set, suggesting high bias.

### 21. Broadly speaking, how does the value of $C$ in logistic regression affect the bias-variance tradeoff? (i.e. As $C$ increases, how are bias and variance affected?)

Answer: as C increases, there is weaker regularization. Overfitting increase, i.e. bias decrease, variance increase

### 22. For your logistic regression models, play around with the regularization hyperparameter, $C$. As you vary $C$, what happens to the fit and coefficients in the model? What do you think this means in the context of this specific problem?

Answer: As C decreases, coefficients become more negative. Bias increase. This means the model has a stronger regularisation strength and can generalize better.  
This also mean that the model will predict left-handedness with less sensitivity to the scores of Q1 to Q44.

### 23. If you have a logistic regression model that has evidence of overfitting, what are three things you might try to do to combat overfitting?

Answer: add more rows, reduce features, increase regularization

---
## Step 6: Answer the problem.

### 24. Suppose you want to understand which psychological features are most important in determining left-handedness. Would you rather use $k$-NN or logistic regression? Why?

Answer: LogReg  

LogReg gives coefficients which can be analyzed to study the 44 questions, on which one influence the probaility the most.

### 25. Select your logistic regression model that utilized LASSO regularization with $\alpha = 1$. Interpret the coefficient for `Q1`.

In [21]:
coef=logreg_models[0].coef_
display(coef.shape)

(1, 44)

Answer: The 44 values are the amount of change to the probability for every unit increase in the question answers.

### 26. If you have to select one model overall to be your *best* model, which model would you select? Why?
- Usually in the "real world," you'll fit many types of models but ultimately need to pick only one! (For example, a client may not understand what it means to have multiple models, or if you're using an algorithm to make a decision, it's probably pretty challenging to use two or more algorithms simultaneously.) It's not always an easy choice, but you'll have to make it soon enough. Pick a model and defend why you picked this model!

Answer: For prediction purpose, I'd pick KNN with $k=25$.  
KNN is a simple 'voting machine' comparing nearest neighbours, whereas LogReg requires linear independence across the 44 questions. It is mathematically more complex too.  

LogReg however is more useful if we want to dissect the changes using a coefficient.


### 27. Circle back to the three specific and conclusively answerable questions you came up with in Q1. Answer one of these for the professor based on the model you selected!

In [28]:
display(max(coef[0,:]))
display(coef[0,3])
display(coef[0,20])
display(coef[0,43])

0.12688402220784448

0.019577084291597834

-0.10459280932507098

0.08413500425777144

Answer:It does appear that Q21: <b>I do not remember birthdays</b> contributes to the probability negatively by 10% with each increase in the score.  
This means that respondants who tend not to remmber birthdays, are likely <b>not</b> to be left-handed

### BONUS:
Looking for more to do? Probably not - you're busy! But if you want to, consider exploring the following. (They could make for a blog post!)
- Create a visual plot comparing training and test metrics for various values of $k$ and various regularization schemes in logistic regression.
- Rather than just evaluating models based on accuracy, consider using sensitivity, specificity, etc.
- In the context of predicting left-handedness, why are unbalanced classes concerning? If you were to re-do this process given those concerns, what changes might you make?
- Fit and evaluate a generalized linear model other than logistic regression (e.g. Poisson regression).
- Suppose this data were in a `SQL` database named `data` and a table named `inventory`. What `SQL` query would return the count of people who were right-handed, left-handed, both, or missing with their class labels of 1, 2, 3, and 0, respectively? (You can assume you've already logged into the database.)